In [2]:
from flask import Flask,jsonify,render_template
from flask_cors import CORS
import json
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine, func, inspect
import datetime as dt

make flask server and do sql stuff

In [39]:
app = Flask(__name__)
CORS(app,resources={r"/*": {"origins": ["http://127.0.0.1:5000"]}})
with open("../restaurant_data.json", 'r') as file:
    data = json.load(file)

engine = create_engine("sqlite:///Resources/votes.sqlite")
base = automap_base()
base.prepare(autoload_with=engine)
# inspector = inspect(engine)
# print(inspector.get_table_names())
types = base.classes.Favorite_Cuisines
# print(f"Reflected table name: {types.__table__.name}")
# print(f"Columns: {types.__table__.columns}")
Session = sessionmaker(bind=engine)
session = Session()
# session.add(types(restaurant_type = "mexican", votes_cast = 0))
# session.add(types(restaurant_type = "italian", votes_cast = 0))
# session.add(types(restaurant_type = "thai", votes_cast = 0))
# session.commit()
testq = session.query(types).all()
print([i.votes_cast for i in testq if True])
session.close()

def pull_vote_totals():
    session = Session()
    boop = session.query(types).all()
    democracy = {"mexican":0, "italian": 0, "thai":0}
    
    for i in boop:
        democracy[i.restaurant_type] = i.votes_cast   
    session.close()
    return democracy
    

def add_vote_to_db(vote,types):
    session = Session()
    tp = session.query(types).filter(types.restaurant_type == vote).first()
    tp.votes_cast += 1
    session.commit()
    session.close()
    return json.dumps("vote added to db")
    

@app.route("/")
def home():
    return render_template('index.html')

@app.route("/map")
def dontwait():
    return render_template('map.html')

@app.route("/chart")
def justkiddingyoucanwait():
    return render_template('chart.html')

@app.route("/btc")
def nvm():
    return render_template('btc.html')

@app.route("/vote<vote>")
def noway(vote):
    add_vote_to_db(vote,types)
    return vote
    
@app.route("/voteTot")
def jose():
    voteTots = pull_vote_totals()
    return json.dumps(voteTots)

@app.route("/data")
def josephine():
    return json.dumps(data)


if(__name__=="__main__"):
    app.run()

[10, 6, 7]
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Apr/2023 16:29:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 16:29:46] "GET /voteitalian HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 16:29:46] "GET /voteTot HTTP/1.1" 200 -


mexican
10
italian
7
thai
7
{'mexican': 10, 'italian': 7, 'thai': 7}


127.0.0.1 - - [13/Apr/2023 16:29:48] "GET /voteTot HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 16:29:48] "GET /voteitalian HTTP/1.1" 200 -


mexican
10
italian
7
thai
7
{'mexican': 10, 'italian': 7, 'thai': 7}


127.0.0.1 - - [13/Apr/2023 16:29:49] "GET /voteitalian HTTP/1.1" 200 -
127.0.0.1 - - [13/Apr/2023 16:29:50] "GET /voteTot HTTP/1.1" 200 -


mexican
10
italian
9
thai
7
{'mexican': 10, 'italian': 9, 'thai': 7}
